In [61]:
import os
import openpyxl
import xlwings as xw
from openpyxl import load_workbook

In [62]:
# %%
def download_pdfs(sample_number, original_excel_file, path_to_store_converted_excel, converted_excel_sheet_name, path_for_pdfs, contract_list):
    """Specify the number of pdfs to download, excel file containing the url links, path for the new converted excel,
    the sheet name you want to extract files from, the path for where you want to store the downloaded pdfs which will be scraped from the sheet,
    and the list of contract IDs you want to download"""
    #Example command line execution:
    # python pdf_download_script.py
    # sample_number:5
    # original_excel_file: 'DTMB_PRO-Contract-List.xls'
    # path_to_store_converted_excel: '/Users/narainyucel/Google Drive/MADS/capstone/legal_nlp'
    # converted_excel_sheet_name: 'Updated 06-16-2022'
    # path_for_pdfs: '/Users/narainyucel/Google Drive/MADS/capstone/legal_nlp'
    # contract_list: ["071B1300128", "210000001036"]
    #convert old xls to xlsx and split them into one xlsx per sheet, choose sheet with relevant information
    with xw.App(visible=False) as app:
        wb = app.books.open(original_excel_file)
        for sheet in wb.sheets:
            wb_new = app.books.add()
            sheet.copy(after=wb_new.sheets[0])
            wb_new.sheets[0].delete()
            if not os.path.isfile(path_to_store_converted_excel + f'/{sheet.name}.xlsx'):
                wb_new.save(path_to_store_converted_excel + f'/{sheet.name}.xlsx')
            wb_new.close()
    #Download pdfs needed
    DOWNLOADS_DIR = path_for_pdfs
    links_list = []
    
    # Extracts all links
    wb1 = load_workbook(path_to_store_converted_excel + f'/{converted_excel_sheet_name}.xlsx')
    ws1 = wb1.get_sheet_by_name(converted_excel_sheet_name)
    total = len(ws1['B'])
    cnt = 2
    for cell in range(1, total):
        try:
            links_list.append(ws1.cell(row=cnt, column=2).hyperlink.target)
            cnt += 1
        except AttributeError:
            links_list.append("No Link")
            cnt += 1
            continue
    
    # Extracts all IDs
    wb = load_workbook(path_to_store_converted_excel + f'/{converted_excel_sheet_name}.xlsx')
    ws = wb.active
    first_column = ws['A']
    ids = [cell.value for cell in first_column[1:]]
    
    # Finds the links associated with the IDs
    found_links = []
    count = 0
    for cell in ids:
        if cell in contract_list:
            found_links.append(links_list[count])
        count += 1
    
    for url in found_links:
        name = url.rsplit('/', 1)[-1]
        filename = os.path.join(DOWNLOADS_DIR, name)
        if not os.path.isfile(filename):
            try:
                urllib.request.urlretrieve(url, filename)
                print(count)
            except HTTPError as error:
                print(error.status, error.reason)
            except URLError as error:
                print(error.reason)
            except TimeoutError:
                print("Request timed out")
    return 0

In [63]:
d = download_pdfs(2, "/Users/ilyasabdulrahman/Desktop/work_docs/DTMB_Contracts/DTMB_PRO-Contract-List.xls", "/Users/ilyasabdulrahman/Desktop/work_docs/DTMB_Contracts", "Updated 06-23-2022", "/Users/ilyasabdulrahman/Desktop/work_docs/DTMB_Contracts", ["210000001420", "071B7700129"])

/var/folders/0p/8kl5kfh54ss3mwwgljmx9bjh0000gn/T/ipykernel_26014/436576433.py:30: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws1 = wb1.get_sheet_by_name(converted_excel_sheet_name)



1497
1497
